Importing necessary packages

In [1]:
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tqdm
from multiprocessing import Pool
import pickle

data_path = '/Users/ruoyangzhang/Documents/PythonWorkingDirectory/news_exploration/Data/'

#### we still need to systematically extract the specific article links

In [2]:
page = requests.get('https://www.reuters.com/resources/archive/us/')

Extract links for specific years

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
links = [link['href'] for link in soup.find_all(href = True) if bool(re.search('/resources/archive/us/20', link['href'])) and len(link['href'].split('.')[-2].split('/')[-1]) == 4]

In [5]:
# completing the links
links = ['https://www.reuters.com' + link for link in links]

In [7]:
links = links[7:]

Now we proceed to extract the aggregated article links in the specific pages

In [9]:
aggregated_article_links = []
for link in tqdm(links):
    page_date = requests.get(link)
    soup_date = BeautifulSoup(page_date.content, 'html.parser')
    links_date = [link['href'] for link in soup_date.find_all(href = True) if bool(re.search('/resources/archive/us/20', link['href'])) and len(link['href'].split('.')[-2].split('/')[-1]) > 4]
    aggregated_article_links.extend(links_date)

100%|██████████| 4/4 [00:01<00:00,  3.76it/s]


In [10]:
# completing the links
aggregated_article_links = ['https://www.reuters.com' + link for link in aggregated_article_links]

Now we extract the article links, we will ignore the non text articles, such as videos

In [11]:
def article_link_extraction(link):
    page_article = requests.get(link)
    soup_article =  BeautifulSoup(page_article.content, 'html.parser')
    article_links = [link['href'] for link in soup_article.find_all(href = True) if 'article' in link['href'].split('/')]
    return(article_links)

In [12]:
p = Pool(8, maxtasksperchild=1)
extraction_res = p.map(article_link_extraction, tqdm(aggregated_article_links))
p.close()

100%|██████████| 1096/1096 [00:00<00:00, 126456.79it/s]


In [13]:
article_links = [item for sublist in extraction_res for item in sublist]

In [25]:
article_links = list(set(article_links))

We save the links for later use

In [15]:
with open(data_path+'article_links.pickle', 'wb') as handle:
    pickle.dump(article_links, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
len(article_links)

1692132

We have collected 8 million article links